In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import utils
from itertools import combinations
from feature_engineering import *
from sklearn.preprocessing import LabelEncoder
import gc
from sklearn.metrics import log_loss
import pickle
from tqdm import tqdm
import re
import urllib

In [2]:
train = pd.read_pickle(FILE.train_ori.value)
test = pd.read_pickle(FILE.test_ori.value)
X = pd.concat([train,test],sort=False)

In [3]:
X_ = X[['instance_id']].copy()

# Process city and province

In [4]:
# def city(x):
#     if len(x) == 1:
#         return x
#     else:
#         return 'c1_{} c2_{} c3_{} c4_{} c5_{}'.format(x[0:3],x[3:6],x[6:9],x[9:12],x[12:15])
    
# def province(x):
#     if len(x) == 1:
#         return x
#     else:
#         return 'p1_{} p2_{} p3_{} p4_{} p5_{}'.format(x[0:3],x[3:6],x[6:9],x[9:12],x[12:15])
    
# X_['city'] = X['city'].astype(str).apply(city)
# X_['province'] = X['province'].astype(str).apply(province)
# le1 = LabelEncoder()
# le1.fit(pd.concat([X_['city'].astype(str),X_['province'].astype(str)]))
# X_['city1'] = le1.transform(X_['city'].astype(str))
# X_['province1'] = le1.transform(X_['province'].astype(str))
# X_['city_province'] = le1.fit_transform(X_['city'].astype(str)+X_['province'].astype(str))

# Process Make and Model

In [5]:
# pattern1=r'%2522|%(25){0,}2B|%20|%(25){0,}28|%(25){0,}29|%(25){0,}2C'
# #用空格替代连字符"-或者+或者_"
# pattern2=r'-{1,}|(?<=[a-zA-Z\d])\+{1,}\s{0,}(?=[a-zA-Z\d])|_{1,}|[/,]'
# #两个以上的空格都替换成一个
# pattern3='\s{2,}'
# #去掉括号
# pattern4='\(|\)'

# def reg(input_string):
#     if isinstance(input_string, str):
#         new_string = input_string.upper()
#         new_string = re.sub(r'IPHONE', r'IPHONE ',new_string)
#         new_string=re.sub(pattern1,' ',new_string)
#         new_string=urllib.parse.unquote(new_string)
#         new_string = re.sub(pattern2,' ',new_string)
#         new_string = re.sub(pattern3,' ', new_string)
#         new_string = re.sub(pattern4,'', new_string)
#         # 全部转为大写
        
#         if '%' in  new_string:
#             print(input_string,new_string)
#             new_string='nan'
#         return new_string
#     else:
#         # print(input_string)
#         return input_string

# X_['make_new']=X['make'].apply(reg)
# X_['model_new']=X['model'].apply(reg)
# # s = X['model'].apply(reg)

In [6]:
def reg(x):
    for m in ['HUAWEI','OPPO', 'iPhone','IPHONE','vivo','HWI','Redmi','huawei','sonny','Changhong','Coolpad','oppo','iphone','GIONEE','redmi','Xplay','changhong']:
        x = re.sub(r'{}'.format(m),r'{} '.format(m),x)
        finds = re.findall(r'%[25]+B',x)
        if len(finds) > 0:
            x = re.sub(finds[0],finds[0]+' ',x)
    return x
X_['make_v1']=X['make'].astype(str).apply(reg)
X_['model_v1']=X['model'].astype(str).apply(reg)

# Model V2 only take care of %, test data 2002

In [7]:
def process_char(x):
    x = urllib.parse.unquote(x)
    x = re.sub(r'%20|%28|%29',' ',x)
    x = re.sub(r'%[25]+B?',' ',x)
    return x

X_['make_v2']=X['make'].astype(str).apply(process_char)
X_['model_v2']=X['model'].astype(str).apply(process_char)

# Model V3  urllib, parse ' '


In [8]:
def process_char(x):
    if x.find('%') != -1:
        index = x.find('%')
        x = x[:index]+' '+ x[index:]
    x = urllib.parse.unquote(x)
    x = re.sub(r'%20|%28|%29',' ',x)
    x = re.sub(r'%[25]+B?',' ',x)
    return x

X_['make_v3']=X['make'].astype(str).apply(process_char)
X_['model_v3']=X['model'].astype(str).apply(process_char)

# Model V4 each word seperate

In [9]:
def v4(x):
    x = ' '.join(x)
    x = re.sub(r' +',' ',x)
    return x
X_['make_v4']=X['make'].astype(str).apply(v4)
X_['model_v4']=X['model'].astype(str).apply(v4)

# Model V5, only touch those has %


In [10]:
def v5(x):
    if x.find('%') == -1:
        return 'nan'
    else:
        x = ' '.join(x)
        x = re.sub(r' +',' ',x)
        return x
X_['make_v5']=X['make'].astype(str).apply(v5)
X_['model_v5']=X['model'].astype(str).apply(v5)

In [11]:


X_.to_pickle('../../data/nn_features/clean.pkl')

In [12]:
s = X_['model_v4'].apply(lambda x: len(x))

In [13]:
X_['model_v4'].iloc[636926]

'O P P O % 2 5 2 5 2 5 2 5 2 5 2 5 2 5 2 B R 9 % 2 5 2 5 2 5 2 5 2 5 2 5 2 5 2 B P l u s t m % 2 5 2 5 2 5 2 5 2 5 2 5 2 5 2 B A'

In [14]:
s.sort_values(ascending=False)

998593     367
250520     367
636926     127
172037     127
828763     127
1000764    127
683962     125
113378     125
172048     125
328271     125
348673     125
32964      125
777464     125
375135     125
402491     125
987670     125
997772     125
532015     125
904442     125
342377     125
94791      125
830087     125
392164     125
66668      125
208286     125
17941      115
43004      115
674622     115
726564     115
163688     115
          ... 
737100       3
737213       3
272565       3
737282       3
736994       3
737336       3
737332       3
272440       3
737520       3
737534       3
272360       3
737596       3
737604       3
272307       3
272794       3
44387        1
547768       1
357573       1
501632       1
136378       1
175105       1
720986       1
575127       1
371225       1
483697       1
142122       1
43321        1
586595       1
411694       1
429391       1
Name: model_v4, Length: 1041674, dtype: int64